In [52]:
import pickle


utube_clickbait_df = pickle.load(open("data/clickbait-df", "rb"))
utube_nonclickbait_df = pickle.load(open("data/non-clickbait-df", "rb"))

In [53]:
utube_clickbait_df.head(2)

,channel_id,channel_name,channel_subscribers,channel_videos,channel_views,video_comments,video_dislikes,video_id,video_likes,video_title,video_views
11218,UC7t6FZ5U17ruEXRH1WRxcTQ,Top List,0,193,143713154,131.0,183.0,hDcWfBq5bxM,1424.0,10 Amazing Places That Disappear During High Tide,360859.0
3168,UCXtjxYo_xhm1GoSPlgfV3eQ,EG Mines,403931,658,119696149,1.0,7.0,D_wROW7u89Y,7.0,Simple Math Test in 59 Seconds - Genius Test,2483.0


In [54]:
utube_clickbait_df.shape

(18317, 11)

In [55]:
utube_clickbait_df.head(2)

,channel_id,channel_name,channel_subscribers,channel_videos,channel_views,video_comments,video_dislikes,video_id,video_likes,video_title,video_views
11218,UC7t6FZ5U17ruEXRH1WRxcTQ,Top List,0,193,143713154,131.0,183.0,hDcWfBq5bxM,1424.0,10 Amazing Places That Disappear During High Tide,360859.0
3168,UCXtjxYo_xhm1GoSPlgfV3eQ,EG Mines,403931,658,119696149,1.0,7.0,D_wROW7u89Y,7.0,Simple Math Test in 59 Seconds - Genius Test,2483.0


In [56]:
utube_nonclickbait_df.shape

(19080, 11)

In [57]:
import re
import emoji
from gensim.parsing.preprocessing import *


def tokenize(string):

    """ Tokenizes a string.
    """
    stop_words = [
        "about", "an", "are", "as", "at", "be", "by", "com", "for", "from", "in", "is", "it", "of", "on", "or", "that",
        "the", "this", "to", "was", "what", "when", "where", "who", "with", "the", "www"
    ]

    string = strip_short(
        strip_multiple_whitespaces(
            strip_punctuation(
                split_alphanum(string))),
        minsize=2)
    
    emojis = [ c for c in string if c in emoji.UNICODE_EMOJI ]
    
    string = stem_text(re.sub(r"[^\w\s,]", "", string))
    

    tokens = string.split() + emojis
    
    for stop_word in stop_words:
        try:
            tokens.remove(stop_word)
        except:
            pass

    return tokens

In [58]:
utube_clickbait_df["video_title_tokenized"] = utube_clickbait_df["video_title"].apply(tokenize)
utube_nonclickbait_df["video_title_tokenized"] = utube_nonclickbait_df["video_title"].apply(tokenize)

In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split


utube_clickbait_df["label"] = 1
utube_nonclickbait_df["label"] = 0

utube_clickbait_df = utube_clickbait_df.sample(frac=1).sample(frac=1).sample(n=18000)
utube_nonclickbait_df = utube_nonclickbait_df.sample(frac=1).sample(frac=1).sample(n=18000)


dataframe = pd.concat([ utube_clickbait_df, utube_nonclickbait_df ]).sample(frac=1).sample(frac=1)

x_utube_train, x_utube_test, y_utube_train, y_utube_test = train_test_split(
    dataframe.loc[:, dataframe.columns != "label"], 
    dataframe["label"], 
    test_size=0.2, 
    random_state=42)

# Export them:
pickle.dump(x_utube_train, open("data/x-train", "wb"))
pickle.dump(y_utube_train, open("data/y-train", "wb"))
pickle.dump(x_utube_test, open("data/x-test", "wb"))
pickle.dump(y_utube_test, open("data/y-test", "wb"))

In [60]:
x_utube_train.shape

(28800, 12)

In [61]:
x_utube_test.shape

(7200, 12)

In [62]:
x_utube_test.head

<bound method NDFrame.head of                      channel_id                        channel_name  \
11158  UCbu2SsF-Or3Rsn3NxqODImw                            GameSpot   
19591  UCq7OHvWO6Z3u-LztFdrcU-g                        Illumination   
18028  UCUHW94eEFW7hkUMVaZz4eDg                       minutephysics   
18673  UCvthuVsurPaVz2a7_4LepGg                     Family Fun Pack   
10489  UCWc3iDmP9rChE3iH1qI3Nqg                         Funny Stuff   
14323  UCHgI_MItRWEKLX7lKz7Xn0g                      Top 5 Unknowns   
19090  UClW44KG_EKETupl-3BlnLSQ                              Top15s   
15136  UC8p1vwvWtl6T73JiExfWs1g                            CBS News   
21588  UCT0hbLDa-unWsnZ6Rjzkfug                  FilmSelect Trailer   
5956   UCHKdYRn_mmCRZ53APbUDxyw                         ViralTop5TH   
21099  UCz1GPotHecuLngiLuY739QQ                  Rotten Tomatoes TV   
16230  UCE-J6hbhHnVJyASqIYcZaAw                           TheTrendy   
4086   UCtNdZrjjurSTFv-JgvQjssw                

In [63]:
import numpy as np


def average_embedding(tokens, word2vec, na_vector=None):

    vectors = list()

    for token in tokens:
        if token in word2vec:
            vectors.append(word2vec[token])

    if len(vectors) == 0 and na_vector is not None:
        vectors.append(na_vector)

    return np.mean(np.array(vectors), axis=0)

In [64]:
import gensim


documents = x_utube_train["video_title_tokenized"]
word2vec = gensim.models.Word2Vec(
    documents,
    size=25,
    window=20,
    min_count=1,
    workers=2
)
word2vec.train(documents, total_examples=len(documents), epochs=30)

# Export it:
pickle.dump(word2vec, open("data/word2vec", "wb"))

In [65]:
titles_embeddings = x_utube_train["video_title_tokenized"].apply(average_embedding, word2vec=word2vec)
utube_training_set = pd.concat(
    [
        x_utube_train[["video_views", "video_likes", "video_dislikes", "video_comments"]],
        titles_embeddings.apply(pd.Series)
    ], axis=1)

utube_training_set["label"] = y_utube_train
utube_training_set = utube_training_set.dropna()

mean_title_embedding = titles_embeddings.dropna().mean(axis=0)
pickle.dump(mean_title_embedding, open("mean-title-embedding", "wb"))

titles_embeddings = x_utube_test["video_title_tokenized"].apply(average_embedding, word2vec=word2vec, na_vector=mean_title_embedding)
utube_testing_set = pd.concat(
    [
        x_utube_test[["video_views", "video_likes", "video_dislikes", "video_comments"]],
        titles_embeddings.apply(pd.Series)
    ], axis=1)
utube_testing_set["label"] = y_utube_test

C:\Users\Abhishek\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if __name__ == '__main__':
C:\Users\Abhishek\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Abhishek\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [66]:
utube_training_set.shape

(28798, 30)

In [67]:
utube_testing_set.shape

(7200, 30)

Considering the logarithm of the video metadata (views, likes, dislikes, comments) to have a normal distribution of values:

In [47]:
utube_training_set[["video_views", "video_likes", "video_dislikes", "video_comments"]] = utube_training_set[["video_views", "video_likes", "video_dislikes", "video_comments"]].apply(np.log)
utube_testing_set[["video_views", "video_likes", "video_dislikes", "video_comments"]] = utube_testing_set[["video_views", "video_likes", "video_dislikes", "video_comments"]].apply(np.log)


utube_training_set = utube_training_set.replace(-np.inf, 0)
utube_testing_set = utube_testing_set.replace(-np.inf, 0)

In [48]:
train_labels = utube_training_set["label"]
test_labels = utube_testing_set["label"]

utube_training_set = utube_training_set.drop(columns=["label"])
utube_testing_set = utube_testing_set.drop(columns=["label"])

pickle.dump(utube_training_set["video_views"].mean(), open("data/mean-log-video-views", "wb"))
pickle.dump(utube_training_set["video_likes"].mean(), open("data/mean-log-video-likes", "wb"))
pickle.dump(utube_training_set["video_dislikes"].mean(), open("data/mean-log-video-dislikes", "wb"))
pickle.dump(utube_training_set["video_comments"].mean(), open("data/mean-log-video-comments", "wb"))

In [49]:
from sklearn import preprocessing


min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(utube_training_set)
utube_training_set = pd.DataFrame(min_max_scaler.transform(utube_training_set), columns=utube_training_set.columns)
utube_testing_set = pd.DataFrame(min_max_scaler.transform(utube_testing_set), columns=utube_testing_set.columns)

# Export it:
pickle.dump(min_max_scaler, open("data/min-max-scaler", "wb"))

In [50]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

In [51]:
svm_params = [
    { "C": np.linspace(1, 25, 10), "gamma": np.linspace(1, 5, 10) },
]

filename = 'data/svm'
grid_search_cv = pickle.load(open(filename, 'rb'))
    

predictions = grid_search_cv.predict(utube_testing_set)

print("Best SVM with:")
print("Performance on the test set (%d samples):" % len(utube_testing_set))
print("\tAccuracy Score:", accuracy_score(test_labels, predictions))
print("\tArea under ROC curve:", roc_auc_score(test_labels, predictions))
print("\tClassification report (on the test set):")
print(classification_report(test_labels, predictions))

#pickle.dump(grid_search_cv.best_estimator_, open("data/svm", "wb"))

Best SVM with:
Performance on the test set (7200 samples):
	Accuracy Score: 0.5809722222222222
	Area under ROC curve: 0.5809722222222222
	Classification report (on the test set):
             precision    recall  f1-score   support

          0       0.55      0.95      0.69      3600
          1       0.80      0.21      0.34      3600

avg / total       0.67      0.58      0.52      7200

